In [1]:
import json
#folder = '/content/drive/MyDrive/project_data/preprocessed_data_no_stemming/'
#file_names = ['Copy of john_f_kennedy_preprocessed2.json']
president_json_file = '/content/drive/MyDrive/project_data/preprocessed_data_no_stemming/Copy of john_f_kennedy_preprocessed2.json'

with open(president_json_file, 'r') as f:
  json_str = f.read()


In [2]:
import pandas as pd

all_text = ''
df_kennedy = pd.DataFrame(columns = ['Text'])
data = json.loads(json_str)
number_of_transcripts = len(data['Items'])
for i in range(5):
  #store in csv as a different record for each transcript
  df_kennedy.loc[len(df_kennedy)] = [data['Items'][i]['transcript']]
  all_text = all_text + data['Items'][i]['transcript']

In [3]:
all_text.replace('\r\n\r\n', ' ')
all_text.replace('\r\n', ' ')

'Mr. Catledge, members of the American Society of Newspaper Editors, ladies and gentlemen:  The President of a great democracy such as ours, and the editors of great newspapers such as yours, owe a common obligation to the people: an obligation to present the facts, to present them with candor, and to present them in perspective. It is with that obligation in mind that I have decided in the last 24 hours to discuss briefly at this time the recent events in Cuba.  On that unhappy island, as in so many other arenas of the contest for freedom, the news has grown worse instead of better. I have emphasized before that this was a struggle of Cuban patriots against a Cuban dictator. While we could not be expected to hide our sympathies, we made it repeatedly clear that the armed forces of this country would not intervene in any way.  Any unilateral American intervention, in the absence of an external attack upon ourselves or an ally, would have been contrary to our traditions and to our inter

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import string
import random
import re

In [5]:


# Example text data
#text = "This is an example text used for training an RNN model for text generation. This is a simple example."
text = all_text

# Preprocess the text by removing punctuation and converting to lowercase
text = text.lower()
text = re.sub(r"[^a-zA-Z\s]", "", text)

# Split the text into words
words = text.split()

# Create a list of unique words and a dictionary to map words to indices
vocab = list(set(words))
word2index = {word: i for i, word in enumerate(vocab)}
index2word = {i: word for i, word in enumerate(vocab)}
vocab_size = len(vocab)

# Convert the text to a sequence of word indices
input_seq = [word2index[word] for word in words]




In [6]:
# Define the RNN model for word-level text generation
class WordRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(WordRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.LSTM(hidden_size, hidden_size)
        #self.rnn = nn.RNN(hidden_size, hidden_size, num_layers = 2, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        output = self.out(output[0])
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size)


In [ ]:
# Initialize the WordRNN model
hidden_size = 100
word_rnn = WordRNN(vocab_size, hidden_size, vocab_size)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(word_rnn.parameters(), lr=0.00001, weight_decay=0.00001)

# Training loop
n_epochs = 5
for epoch in range(n_epochs):
    for i in range(len(input_seq) - 1):
        input_tensor = torch.tensor([[input_seq[i]]], dtype=torch.long)
        target_tensor = torch.tensor([input_seq[i + 1]], dtype=torch.long)

        hidden = word_rnn.init_hidden()
        optimizer.zero_grad()

        loss = 0
        for j in range(len(input_tensor)):
            output, hidden = word_rnn(input_tensor[j], hidden)
            loss += criterion(output, target_tensor)

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {loss.item()}")



In [33]:
optimizer = optim.Adam(word_rnn.parameters(), lr=0.000006, weight_decay=0.00001)
# Training loop
n_epochs = 5
for epoch in range(n_epochs):
    for i in range(len(input_seq) - 1):
        input_tensor = torch.tensor([[input_seq[i]]], dtype=torch.long)
        target_tensor = torch.tensor([input_seq[i + 1]], dtype=torch.long)

        hidden = word_rnn.init_hidden()
        optimizer.zero_grad()

        loss = 0
        for j in range(len(input_tensor)):
            output, hidden = word_rnn(input_tensor[j], hidden)
            loss += criterion(output, target_tensor)

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {loss.item()}")

Epoch 1/5, Loss: 8.172811508178711
Epoch 2/5, Loss: 8.087301254272461
Epoch 3/5, Loss: 8.016680717468262
Epoch 4/5, Loss: 8.016435623168945
Epoch 5/5, Loss: 8.110291481018066


learning rate: 0.001
hidden_size = 100
10 epochs => accuracy doesn't improve after 5

learning rate: 0.0001
hidden_size = 200
5 epochs

In [40]:
# Generate text using the trained WordRNN model
def generate_text_word_rnn(model, start_words_str, max_length=100):
    start_words = start_words_str.split()

    with torch.no_grad():
        input = torch.tensor([[word2index[word.lower()] for word in start_words]], dtype=torch.long)
        hidden = model.init_hidden()
        output_text = ' '.join(start_words)

        for _ in range(max_length):
            output, hidden = model(input, hidden)
            _, topi = output.topk(1)
            topi = topi[0][0].item()
            if index2word[topi] == "<eos>":
                break
            else:
                word = index2word[topi]
                output_text += " " + word
                input = torch.tensor([[topi]], dtype=torch.long)

        return output_text




In [41]:
prompt = "American"
# Generate text using the trained WordRNN model
generated_text = generate_text_word_rnn(word_rnn, start_words_str=prompt, max_length=500)
print(generated_text)

American the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the united to the united the u

In [11]:
output_df = pd.DataFrame(columns = ['Parameters', 'Prompt', 'Generated speech'])

In [42]:
output_df.loc[len(output_df)] = ['learning rate: 0.00001, hidden_size = 150, 5 epochs', 'American', generated_text]
#, added a layer of Vanilla RNN with num_layers=5

In [37]:
output_df.shape

(4, 3)

In [43]:
output_df.to_csv('kennedy_rnn.csv')